In [1]:
import pandas as pd
from optbinning import OptimalBinning
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np

/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


(CVXPY) Feb 07 08:32:49 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Feb 07 08:32:49 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
file_path = "/hdd1/sams/diabetes/diabetes.arff"
with open(file_path, 'r') as f:
    lines = f.readlines()

In [3]:
flag = False
rows = []
for line in lines:
    if line.strip() == "@data":
        flag = True
        continue
    
    if not flag:
        continue
    
    row = [i for i in line.strip().split(',')]
    rows.append(row) 

In [4]:
data = np.array(rows)
df = pd.DataFrame(data)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2278392,8222157,'Caucasian','Female','[0-10)','?',6,25,1,1,...,'No','No','No','No','No','No','No','No','No','NO'
1,149190,55629189,'Caucasian','Female','[10-20)','?',1,1,7,3,...,'No','Up','No','No','No','No','No','Ch','Yes','>30'
2,64410,86047875,'AfricanAmerican','Female','[20-30)','?',1,1,7,2,...,'No','No','No','No','No','No','No','No','Yes','NO'
3,500364,82442376,'Caucasian','Male','[30-40)','?',1,1,7,2,...,'No','Up','No','No','No','No','No','Ch','Yes','NO'
4,16680,42519267,'Caucasian','Male','[40-50)','?',1,1,7,1,...,'No','Steady','No','No','No','No','No','Ch','Yes','NO'


In [6]:
y_map = {"'NO'":0, "'<30'":1, "'>30'":1}
df[49] = df[49].map(y_map)

In [7]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2278392,8222157,'Caucasian','Female','[0-10)','?',6,25,1,1,...,'No','No','No','No','No','No','No','No','No',0
1,149190,55629189,'Caucasian','Female','[10-20)','?',1,1,7,3,...,'No','Up','No','No','No','No','No','Ch','Yes',1
2,64410,86047875,'AfricanAmerican','Female','[20-30)','?',1,1,7,2,...,'No','No','No','No','No','No','No','No','Yes',0
3,500364,82442376,'Caucasian','Male','[30-40)','?',1,1,7,2,...,'No','Up','No','No','No','No','No','Ch','Yes',0
4,16680,42519267,'Caucasian','Male','[40-50)','?',1,1,7,1,...,'No','Steady','No','No','No','No','No','Ch','Yes',0


In [8]:
patient_col = df[1].values
d = {}
for i in patient_col:
    if i not in d:
        d[i] = 0
    d[i] += 1

In [9]:
map_d = {}
for k,v in d.items():
    if v > 1:
        map_d[k] = 1
    else:
        map_d[k] = 0

In [10]:
# df[1] = df[1].map(d)
df[1] = df[1].map(map_d)

In [12]:
# diag_1 18
print(len(df[18].unique()))
y = df[49].values
optb = OptimalBinning(name="1", dtype="categorical", solver="cp",
                      monotonic_trend="auto", max_n_prebins=100,
                      min_prebin_size=0.001,time_limit=200)
optb.fit(df[18].values,y)
print(optb.status)

x = optb.transform(df[18], metric="indices")
print(max(x))
df[18] = x

717
OPTIMAL
90


In [13]:
# diag_2 19
print(len(df[19].unique()))
y = df[49].values
optb = OptimalBinning(name="1", dtype="categorical", solver="cp",
                      monotonic_trend="auto", max_n_prebins=100,
                      min_prebin_size=0.001,time_limit=200)
optb.fit(df[19].values,y)
print(optb.status)
x = optb.transform(df[19], metric="indices")
print(max(x))
df[19] = x

749
OPTIMAL
79


In [14]:
# diag_3 20
print(len(df[20].unique()))
y = df[49].values
optb = OptimalBinning(name="1", dtype="categorical", solver="cp",
                      monotonic_trend="auto", max_n_prebins=100,
                      min_prebin_size=0.001,time_limit=200)
optb.fit(df[20].values,y)
print(optb.status)
x = optb.transform(df[20], metric="indices")
print(max(x))
df[20] = x

790
OPTIMAL
82


In [15]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2278392,0,'Caucasian','Female','[0-10)','?',6,25,1,1,...,'No','No','No','No','No','No','No','No','No',0
1,149190,0,'Caucasian','Female','[10-20)','?',1,1,7,3,...,'No','Up','No','No','No','No','No','Ch','Yes',1
2,64410,0,'AfricanAmerican','Female','[20-30)','?',1,1,7,2,...,'No','No','No','No','No','No','No','No','Yes',0
3,500364,0,'Caucasian','Male','[30-40)','?',1,1,7,2,...,'No','Up','No','No','No','No','No','Ch','Yes',0
4,16680,0,'Caucasian','Male','[40-50)','?',1,1,7,1,...,'No','Steady','No','No','No','No','No','Ch','Yes',0


In [16]:
del df[0]

In [17]:
Y = df[49]
del df[49]
X = df

In [18]:
# tokenization
col_to_value2token = {}
cnt = 0
for col in X.columns:
    col_values = X[col].values
    col_to_value2token[col] = {}
    col_values_set = set(col_values)
    for val in col_values_set:
        if val not in col_to_value2token[col]:
            col_to_value2token[col][val] = cnt
            cnt += 1

In [20]:
len(col_to_value2token[19])

80

In [21]:
print(cnt)

833


In [22]:
for col in X.columns:
    value_to_token = col_to_value2token[col]
    X[col] = X[col].map(value_to_token)

In [23]:
df = pd.concat([Y, X], axis = 1)
file_path = "/hdd1/sams/credit/credit.csv"
df.to_csv(file_path, index = False, header = False)

In [36]:
# split test, val and train
from sklearn.model_selection import train_test_split

X_train, X_tmp, Y_train, Y_tmp = train_test_split(X, Y, test_size=0.2, random_state=2024)
X_val, X_test, Y_val, Y_test = train_test_split(X_tmp, Y_tmp, test_size=0.5, random_state=2024)

In [39]:
# save to libsvm
def save_dataset(X, Y, file_path:str):
    with open(file_path, 'w', encoding='utf-8') as f:
        n = len(X)
        for i in range(n):
            row = X.iloc[i].values 
            y = Y.iloc[i]
            items =[str(y)] + [str(i)+":1" for i in row]
            line = ' '.join(items) + '\n'
            f.write(line)

In [40]:
train_file = '/hdd1/sams/data/diabetes/train.libsvm'
val_file = '/hdd1/sams/data/diabetes/val.libsvm'
test_file = '/hdd1/sams/data/diabetes/test.libsvm'
save_dataset(X_train, Y_train, train_file)
save_dataset(X_val, Y_val, val_file)
save_dataset(X_test, Y_test, test_file)

In [27]:
print(f"train: POS/ALL -> {sum(Y_train.values.reshape(-1))}/{len(Y_train)}")
print(f"val: POS/ALL -> {sum(Y_val.values.reshape(-1))}/{len(Y_val)}")
print(f"test: POS/ALL -> {sum(Y_test.values.reshape(-1))}/{len(Y_test)}")

train: POS/ALL -> 37438/81412
val: POS/ALL -> 4692/10177
test: POS/ALL -> 4772/10177


In [33]:
Y_train.iloc[0]

1